In [ ]:
import os
import magic
from PIL import Image
from collections import defaultdict # 확장자 개수를 쉽게 세기 위해 defaultdict 사용

# 검사할 최상위 폴더 경로
target_directory = '../데이터/data/kfood'

# 확장자별 개수를 저장할 딕셔너리
# defaultdict(int)는 키가 없을 경우 자동으로 0을 기본값으로 설정해줍니다.
extension_counts = defaultdict(int)

# 다양한 파일 형식의 MIME 타입과 확장자 매핑
mime_to_ext = {
    'image/jpeg': '.jpeg', # .jpg도 포함
    'image/png': '.png',
    'image/gif': '.gif',
    'image/bmp': '.bmp',
    'image/webp': '.webp',
    'image/tiff': '.tiff',
    'image/svg+xml': '.svg', # SVG 벡터 이미지
    'image/heic': '.heic',   # HEIF (High Efficiency Image Format)
    'image/heif': '.heif',
    'image/mpo' : '.mpo'
}

# 문제 파일 개수를 세기 위한 변수
problem_file_count = 0
total_file_count = 0
header_printed = False

# 폴더 존재 여부 확인
if not os.path.exists(target_directory):
    print(f"오류: '{target_directory}' 폴더를 찾을 수 없습니다.")
else:
    print("파일 검사를 시작합니다...")
    # os.walk를 사용하여 모든 하위 폴더와 파일 순회
    for dirpath, _, filenames in os.walk(target_directory):
        for filename in filenames:
            total_file_count += 1
            file_path = os.path.join(dirpath, filename)
            relative_path = os.path.relpath(file_path, target_directory)

            try:
                _, file_extension = os.path.splitext(filename)
                file_extension = file_extension.lower()

                # 확장자 개수 집계 (확장자가 없는 파일은 '.no_ext'로 처리)
                if file_extension:
                    extension_counts[file_extension] += 1
                else:
                    extension_counts['.no_ext'] += 1

                mime_type = magic.from_file(file_path, mime=True)

                is_problem = False
                message = ""

                # 1. JPG/JPEG 파일 검증
                if mime_type == 'image/jpeg':
                    try:
                        with Image.open(file_path) as img:
                            img.verify()
                        continue
                    except Exception as e:
                        is_problem = True
                        message = f"🚨 {relative_path:<58} | {'⚠️ TF 오류 가능':<13} | 손상된 JPEG 파일: {e}"

                # 2. 다른 파일 형식 처리
                else:
                    expected_ext = mime_to_ext.get(mime_type)
                    is_jpeg_match = (expected_ext == '.jpeg' and file_extension in ['.jpg', '.jpeg'])

                    if expected_ext and (is_jpeg_match or file_extension == expected_ext):
                        continue
                    else:
                        is_problem = True
                        actual_type_info = mime_to_ext.get(mime_type, mime_type)
                        message = f"   {relative_path:<57} | {'불일치':<15} | 확장자({file_extension}), 실제({actual_type_info})"

            except Exception as e:
                is_problem = True
                message = f"🚨 {relative_path:<58} | {'오류':<15} | 파일 처리 중 오류 발생: {e}"

            if is_problem:
                if not header_printed:
                    print("-" * 100)
                    print(f"{'파일 경로':<60} | {'상태':<15} | {'상세 내용'}")
                    print("-" * 100)
                    header_printed = True
                print(message)
                problem_file_count += 1

    # 최종 결과 출력
    print("\n" + "="*40)
    print("       종합 검사 결과")
    print("="*40)

    if problem_file_count == 0:
        print(f"\n🎉 총 {total_file_count}개의 파일을 검사했으며, 문제가 발견되지 않았습니다.")
    else:
        print(f"\n- 총 {total_file_count}개의 파일 중 {problem_file_count}개의 문제 파일을 발견했습니다.")

    # 확장자별 개수 출력
    if extension_counts:
        print("\n--- 확장자별 파일 개수 ---")
        # 개수를 기준으로 내림차순 정렬하여 출력
        sorted_extensions = sorted(extension_counts.items(), key=lambda item: item[1], reverse=True)
        for ext, count in sorted_extensions:
            print(f"  {ext:<15} : {count} 개")
    print("="*40)

In [29]:
import os

# ⭐️ 파일을 삭제할 최상위 폴더 경로 (중요: 경로를 정확히 확인하세요!)
target_directory = '../데이터/data/kfood'

# ⭐️ 삭제할 파일 확장자 목록 (소문자로 작성)
# extensions_to_delete = ['.csv', '.properties']
# extensions_to_delete = ['.png', '.bmp', '.gif']
extensions_to_delete = ['.DS_Store']
# 삭제된 파일과 실패한 파일 수를 세기 위한 변수
deleted_count = 0
failed_count = 0

# 폴더 존재 여부 확인
if not os.path.exists(target_directory):
    print(f"🚨 오류: '{target_directory}' 폴더를 찾을 수 없습니다. 경로를 확인해주세요.")
else:
    print(f"'{target_directory}' 폴더 및 하위 폴더에서 파일 삭제를 시작합니다...")

    # os.walk를 사용하여 모든 하위 폴더와 파일 순회
    for dirpath, _, filenames in os.walk(target_directory):
        for filename in filenames:
            # 파일의 전체 경로 생성
            file_path = os.path.join(dirpath, filename)

            # 파일의 확장자 추출 및 소문자 변환
            _, file_extension = os.path.splitext(filename)
            file_extension = file_extension.lower()

            # 1. 삭제할 확장자 목록에 포함되거나 (or)
            # 2. 확장자가 없는 경우 ('.no_ext')
            if file_extension in extensions_to_delete or not file_extension:
                try:
                    # 파일 삭제
                    os.remove(file_path)
                    print(f"🗑️ 삭제 완료: {file_path}")
                    deleted_count += 1
                except OSError as e:
                    print(f"❌ 삭제 실패: {file_path} (오류: {e})")
                    failed_count += 1

    # 최종 결과 출력
    print("\n" + "="*40)
    print("         파일 삭제 결과 요약")
    print("="*40)
    print(f"총 {deleted_count}개의 파일을 삭제했습니다.")
    if failed_count > 0:
        print(f"총 {failed_count}개의 파일 삭제에 실패했습니다.")
    print("="*40)

'../데이터/data/kfood' 폴더 및 하위 폴더에서 파일 삭제를 시작합니다...
🗑️ 삭제 완료: ../데이터/data/kfood/.DS_Store

         파일 삭제 결과 요약
총 1개의 파일을 삭제했습니다.


In [24]:
import os
from PIL import Image

# ⭐️ 이미지를 변환할 최상위 폴더 경로
target_directory = '../데이터/data/kfood'

# ⭐️ 변환 대상이 될 파일 확장자 목록 (소문자로)
source_extensions = ['.png', '.gif', '.bmp']

# 변환 및 오류 카운트
converted_count = 0
error_count = 0

print(f"'{target_directory}' 폴더 및 하위 폴더에서 이미지 변환을 시작합니다...")

# 모든 하위 폴더와 파일 순회
for dirpath, _, filenames in os.walk(target_directory):
    for filename in filenames:
        # 파일 확장자 추출 및 소문자 변환
        _, file_extension = os.path.splitext(filename)
        file_extension = file_extension.lower()

        # 변환 대상 확장자인지 확인
        if file_extension in source_extensions:
            original_path = os.path.join(dirpath, filename)

            try:
                # 이미지 열기
                with Image.open(original_path) as img:
                    # 1. 이미지를 3채널 RGB 모드로 변환
                    rgb_img = img.convert('RGB')

                    # 2. 새 파일명 생성 (확장자를 .jpg로 변경)
                    basename = os.path.splitext(filename)[0]
                    new_filename = basename + '.jpg'
                    new_path = os.path.join(dirpath, new_filename)

                    # 3. (핵심) 이름 충돌 방지 로직
                    counter = 1
                    # 동일한 이름의 파일이 존재하면, 새 이름 생성
                    while os.path.exists(new_path):
                        new_filename = f"{basename}_{counter}.jpg"
                        new_path = os.path.join(dirpath, new_filename)
                        counter += 1

                    # 4. RGB 이미지 저장
                    rgb_img.save(new_path, 'JPEG')

                    # 파일 이름이 변경되었는지 확인하여 로그 출력
                    original_new_name = basename + '.jpg'
                    if new_filename != original_new_name:
                        print(f"⚠️ 이름 변경 후 변환 완료: {filename} -> {new_filename}")
                    else:
                        print(f"✅ 변환 완료: {filename} -> {new_filename}")

                    converted_count += 1

            except Exception as e:
                print(f"❌ 변환 오류: {original_path} (오류: {e})")
                error_count += 1

# 최종 결과 출력
print("\n" + "="*40)
print("         이미지 변환 결과 요약")
print("="*40)
print(f"총 {converted_count}개의 파일을 JPG로 변환했습니다.")
if error_count > 0:
    print(f"총 {error_count}개의 파일에서 변환 오류가 발생했습니다.")
print("="*40)

'../데이터/data/kfood' 폴더 및 하위 폴더에서 이미지 변환을 시작합니다...
✅ 변환 완료: Img_020_0999.png -> Img_020_0999.jpg
✅ 변환 완료: Img_020_0033.png -> Img_020_0033.jpg
✅ 변환 완료: Img_006_0056.png -> Img_006_0056.jpg
✅ 변환 완료: Img_006_0981.png -> Img_006_0981.jpg
✅ 변환 완료: Img_006_0013.png -> Img_006_0013.jpg
✅ 변환 완료: Img_123_0045.gif -> Img_123_0045.jpg
✅ 변환 완료: Img_123_0147.gif -> Img_123_0147.jpg
✅ 변환 완료: Img_123_0088.bmp -> Img_123_0088.jpg
✅ 변환 완료: Img_116_0008.gif -> Img_116_0008.jpg
✅ 변환 완료: Img_116_0009.gif -> Img_116_0009.jpg
✅ 변환 완료: Img_116_0007.gif -> Img_116_0007.jpg
✅ 변환 완료: Img_116_0010.gif -> Img_116_0010.jpg
✅ 변환 완료: Img_116_0011.gif -> Img_116_0011.jpg
✅ 변환 완료: Img_047_0783.gif -> Img_047_0783.jpg
✅ 변환 완료: Img_047_0058.png -> Img_047_0058.jpg
✅ 변환 완료: Img_047_0032.gif -> Img_047_0032.jpg
✅ 변환 완료: Img_047_0066.png -> Img_047_0066.jpg
✅ 변환 완료: Img_047_0074.png -> Img_047_0074.jpg
✅ 변환 완료: Img_047_0039.png -> Img_047_0039.jpg
✅ 변환 완료: Img_047_0773.png -> Img_047_0773.jpg
✅ 변환 완료: Img_047_0798.png -> I

/opt/anaconda3/envs/stenv/lib/python3.9/site-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


✅ 변환 완료: Img_028_0317.png -> Img_028_0317.jpg
✅ 변환 완료: Img_084_0136.gif -> Img_084_0136.jpg
✅ 변환 완료: Img_103_0066.gif -> Img_103_0066.jpg
✅ 변환 완료: Img_103_0988.png -> Img_103_0988.jpg
✅ 변환 완료: Img_103_0005.png -> Img_103_0005.jpg
✅ 변환 완료: Img_025_0999.png -> Img_025_0999.jpg
✅ 변환 완료: Img_025_0998.png -> Img_025_0998.jpg
✅ 변환 완료: Img_025_0013.png -> Img_025_0013.jpg
✅ 변환 완료: Img_025_0017.png -> Img_025_0017.jpg
✅ 변환 완료: Img_025_0016.png -> Img_025_0016.jpg
✅ 변환 완료: Img_025_0996.png -> Img_025_0996.jpg
✅ 변환 완료: Img_025_0014.png -> Img_025_0014.jpg
✅ 변환 완료: Img_025_0015.png -> Img_025_0015.jpg
✅ 변환 완료: Img_025_0997.png -> Img_025_0997.jpg
✅ 변환 완료: Img_052_0107.gif -> Img_052_0107.jpg
✅ 변환 완료: Img_052_0106.gif -> Img_052_0106.jpg
✅ 변환 완료: Img_052_0971.png -> Img_052_0971.jpg
✅ 변환 완료: Img_052_0105.gif -> Img_052_0105.jpg
✅ 변환 완료: Img_052_0084.png -> Img_052_0084.jpg
✅ 변환 완료: Img_052_0940.bmp -> Img_052_0940.jpg
✅ 변환 완료: Img_052_0519.png -> Img_052_0519.jpg
✅ 변환 완료: Img_052_0021.bmp -> Img_0

In [26]:
import os
from PIL import Image
import magic

# ⭐️ 이미지를 변환할 최상위 폴더 경로
target_directory = '../데이터/data/kfood'

# 변환 및 오류 카운트
converted_count = 0
error_count = 0

print(f"'{target_directory}' 폴더에서 모든 비(非)JPEG 이미지 변환을 시작합니다...")
print("실제 파일 형식을 기준으로 변환하며, 성공 시 원본은 삭제됩니다.")

# 모든 하위 폴더와 파일 순회
for dirpath, _, filenames in os.walk(target_directory):
    for filename in filenames:
        original_path = os.path.join(dirpath, filename)

        try:
            # 1. (핵심) 파일의 실제 MIME 타입 확인
            mime_type = magic.from_file(original_path, mime=True)

            # 2. 실제 JPEG가 아니면 모두 변환 대상으로 지정
            if mime_type != 'image/jpeg':
                # 이미지 열기
                with Image.open(original_path) as img:
                    rgb_img = img.convert('RGB')

                    basename = os.path.splitext(filename)[0]
                    new_filename = basename + '.jpg'
                    new_path = os.path.join(dirpath, new_filename)

                    # 이름 충돌 방지 로직
                    counter = 1
                    while os.path.exists(new_path):
                        # 자기 자신을 변환하는 경우 (예: 'a.jpg'(PNG) -> 'a.jpg'(JPEG)) 무한루프 방지
                        if new_path == original_path:
                            break
                        new_filename = f"{basename}_{counter}.jpg"
                        new_path = os.path.join(dirpath, new_filename)
                        counter += 1

                    # 변환된 이미지 저장
                    rgb_img.save(new_path, 'JPEG')

                # 원본 파일 삭제 (자기 자신을 덮어쓴 경우는 제외)
                if original_path != new_path:
                    os.remove(original_path)

                print(f"✅ 변환 완료: {filename} ({mime_type}) -> {new_filename}")
                converted_count += 1

        except Exception as e:
            # magic이나 PIL에서 이미지 파일이 아니라고 판단하는 경우 등
            if "cannot identify image file" not in str(e): # 일반적인 오류만 출력
                 print(f"❌ 처리 오류: {filename} (오류: {e})")
                 error_count += 1

# 최종 결과 출력
print("\n" + "="*50)
print("           최종 변환 결과 요약")
print("="*50)
print(f"총 {converted_count}개의 파일을 JPG로 변환/통일했습니다.")
if error_count > 0:
    print(f"총 {error_count}개의 파일에서 처리 오류가 발생했습니다.")
print("="*50)

'../데이터/data/kfood' 폴더에서 모든 비(非)JPEG 이미지 변환을 시작합니다...
실제 파일 형식을 기준으로 변환하며, 성공 시 원본은 삭제됩니다.
✅ 변환 완료: Img_020_0969.jpg (image/png) -> Img_020_0969.jpg
✅ 변환 완료: Img_020_0003.jpg (image/png) -> Img_020_0003.jpg
✅ 변환 완료: Img_020_0266.jpg (image/png) -> Img_020_0266.jpg
✅ 변환 완료: Img_020_0310.jpg (image/png) -> Img_020_0310.jpg
✅ 변환 완료: Img_020_0274.jpg (image/gif) -> Img_020_0274.jpg
✅ 변환 완료: Img_020_0114.jpg (image/png) -> Img_020_0114.jpg
✅ 변환 완료: Img_020_0291.jpg (image/png) -> Img_020_0291.jpg
✅ 변환 완료: Img_020_0254.jpg (image/png) -> Img_020_0254.jpg
✅ 변환 완료: Img_020_0321.jpg (image/gif) -> Img_020_0321.jpg
✅ 변환 완료: Img_020_0323.jpg (image/gif) -> Img_020_0323.jpg
✅ 변환 완료: Img_020_0144.jpg (image/png) -> Img_020_0144.jpg
✅ 변환 완료: Img_020_0192.jpg (image/png) -> Img_020_0192.jpg
✅ 변환 완료: Img_020_0182.jpg (image/png) -> Img_020_0182.jpg
✅ 변환 완료: Img_020_0220.jpg (image/png) -> Img_020_0220.jpg
✅ 변환 완료: Img_005_0101.jpg (image/png) -> Img_005_0101.jpg
✅ 변환 완료: Img_005_0147.jpg (image/png) -

In [ ]:
import os
import magic

# ⭐️ 검사할 최상위 폴더 경로
target_directory = '../데이터/data/kfood'

imposter_files = []

print(f"'{target_directory}' 폴더에서 확장자와 실제 형식이 다른 파일을 찾습니다...")

if not os.path.exists(target_directory):
    print(f"🚨 오류: '{target_directory}' 폴더를 찾을 수 없습니다.")
else:
    for dirpath, _, filenames in os.walk(target_directory):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            _, file_extension = os.path.splitext(filename)
            file_extension = file_extension.lower()

            # 확장자가 .jpg 또는 .jpeg인 파일만 대상으로 검사
            if file_extension in ['.jpg', '.jpeg']:
                try:
                    # 실제 파일의 MIME 타입 확인
                    mime_type = magic.from_file(file_path, mime=True)

                    # 확장자는 JPG인데, 실제 타입이 PNG인 경우
                    if mime_type == 'image/png':
                        imposter_files.append(file_path)

                except Exception as e:
                    print(f"⚠️ 검사 오류: {file_path} ({e})")

    # --- 최종 결과 출력 ---
    print("\n" + "="*50)
    if not imposter_files:
        print("✅ 검사 완료. 확장자와 실제 형식이 다른 JPG/JPEG 파일이 없습니다.")
    else:
        print(f"총 {len(imposter_files)}개의 '위장' 파일을 찾았습니다.")
        print("이 파일들은 확장자가 .jpg/.jpeg이지만, 실제로는 PNG 파일입니다.")
        print("--- 문제 파일 목록 ---")
        for path in imposter_files:
            print(path)
    print("="*50)

In [ ]:
import os
import glob
import tensorflow as tf
from tqdm import tqdm

# --- 설정 ---
# ▼▼▼ 검사할 데이터 폴더의 경로를 입력하세요 ▼▼▼
DATA_PATH = '../데이터/data/kfood'

# --- 메인 실행 로직 ---
print(f"TensorFlow 기반 데이터셋 무결성 검사를 시작합니다...")
print(f"대상 경로: {os.path.abspath(DATA_PATH)}")

all_files = glob.glob(os.path.join(DATA_PATH, '**', '*'), recursive=True)
all_files = [f for f in all_files if os.path.isfile(f)]

# 문제가 있는 파일 경로를 저장할 리스트
problematic_files = []

if not all_files:
    print("검사할 파일이 없습니다.")
else:
    for file_path in tqdm(all_files, desc="파일 검사 중"):
        try:
            # 1. TensorFlow로 파일을 바이트 단위로 읽기
            img_bytes = tf.io.read_file(file_path)

            # 2. TensorFlow로 이미지 디코딩 시도 (학습 시와 동일한 방식)
            # 이 부분이 실패하면 학습 시에도 실패합니다.
            tf.io.decode_image(img_bytes)

        except tf.errors.InvalidArgumentError as e:
            # TensorFlow가 이미지로 인식하지 못하는 경우
            if "Unknown image file format" in str(e):
                problematic_files.append({
                    "path": file_path,
                    "reason": "TensorFlow가 인식할 수 없는 이미지 형식 또는 손상된 파일입니다."
                })
        except Exception as e:
            # 기타 예외 처리
            problematic_files.append({
                "path": file_path,
                "reason": f"알 수 없는 오류: {str(e)}"
            })

# --- 결과 출력 ---
print("\n" + "=" * 50)
if not problematic_files:
    print("✅ 모든 파일을 성공적으로 확인했습니다. TensorFlow 호환성 문제가 발견되지 않았습니다.")
else:
    print(f"⚠️ 총 {len(problematic_files)}개의 문제 파일을 찾았습니다.")
    print("--- 문제 파일 목록 ---")
    for item in problematic_files:
        print(f"파일: {item['path']}")
        print(f"  원인: {item['reason']}")
        print("-" * 20)

    print("\n위 파일들을 삭제하거나 수정 후 다시 시도해 주세요.")
print("=" * 50)

In [1]:
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm

# ❗❗ 본인의 이미지 데이터셋 상위 폴더 경로로 수정해주세요.
data_dir = Path('../데이터/data/kfood')

# 처리할 이미지 파일 확장자
image_extensions = ['.jpg', '.jpeg', '.png']

print(f"'{data_dir}' 폴더의 모든 이미지를 재저장합니다.")
print("이 작업은 매우 오래 걸릴 수 있습니다. 원본 데이터는 반드시 백업해두세요.")

# 전체 파일 목록을 먼저 수집
all_files = [p for p in data_dir.rglob('*') if p.suffix.lower() in image_extensions]

if not all_files:
    print("처리할 이미지 파일을 찾지 못했습니다.")
else:
    success_count = 0
    fail_count = 0

    # tqdm을 사용하여 진행률 표시
    for file_path in tqdm(all_files, desc="이미지 재저장 중"):
        try:
            # 이미지를 열고 RGB 모드로 변환 (투명도 채널 등 제거하여 일관성 확보)
            img = Image.open(file_path).convert('RGB')

            # 원본 파일을 덮어쓰며 JPEG 형식으로 저장
            # quality=95 옵션으로 원본과 유사한 품질 유지
            img.save(file_path, 'jpeg', quality=95)
            success_count += 1
        except Exception as e:
            # 심하게 손상되어 열 수 없는 파일은 실패 처리
            print(f"\n처리 실패: {file_path} | 원인: {e}")
            fail_count += 1

    print("\n--- 작업 완료 ---")
    print(f"총 {success_count}개의 파일을 성공적으로 재저장했습니다.")
    if fail_count > 0:
        print(f"총 {fail_count}개의 파일은 처리 중 오류가 발생했습니다.")

'../데이터/data/kfood' 폴더의 모든 이미지를 재저장합니다.
이 작업은 매우 오래 걸릴 수 있습니다. 원본 데이터는 반드시 백업해두세요.


이미지 재저장 중:  45%|████▌     | 67982/150610 [02:08<02:32, 543.19it/s]/opt/anaconda3/envs/stenv/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
이미지 재저장 중: 100%|██████████| 150610/150610 [04:32<00:00, 553.45it/s]


--- 작업 완료 ---
총 150610개의 파일을 성공적으로 재저장했습니다.


In [28]:
import os
from PIL import Image
from tqdm import tqdm

# --- 설정 ---
# ▼▼▼ 검사할 데이터 폴더의 경로를 입력하세요 ▼▼▼
TARGET_DIRECTORY = '../데이터/data/kfood'

# --- 함수 정의 ---
def get_all_image_paths(directory):
    """지정된 디렉토리와 하위 디렉토리의 모든 파일 경로를 리스트로 반환합니다."""
    all_files = []
    # os.walk를 사용하여 모든 하위 폴더 탐색
    for dirpath, _, filenames in os.walk(directory):
        for filename in filenames:
            all_files.append(os.path.join(dirpath, filename))
    return all_files

# --- 메인 실행 로직 ---
print(f"이미지 무결성 검사를 시작합니다...")
print(f"대상 경로: {os.path.abspath(TARGET_DIRECTORY)}")

# 1. 검사할 모든 파일 목록 가져오기
try:
    image_paths = get_all_image_paths(TARGET_DIRECTORY)
    if not image_paths:
        print("검사할 파일이 없습니다.")
        exit()
except FileNotFoundError:
    print(f"오류: '{TARGET_DIRECTORY}' 경로를 찾을 수 없습니다.")
    exit()

# 2. 문제가 있는 파일을 저장할 리스트
bad_files = []

# 3. tqdm을 사용하여 진행 상황을 보며 각 파일 검사
for file_path in tqdm(image_paths, desc="파일 검사 중"):
    try:
        # (1) 이미지 파일 열기
        with Image.open(file_path) as img:
            # (2) 이미지 헤더 및 기본 구조 검증 (빠른 검사)
            img.verify()

        # (3) 이미지를 완전히 메모리에 로드하여 데이터 손상 여부 확인 (정밀 검사)
        # verify() 후에는 파일을 다시 열어야 합니다.
        with Image.open(file_path) as img:
            img.load()

    except Exception as e:
        # 파일 열기, 검증, 로드 중 어느 단계에서든 예외가 발생하면 문제 파일로 간주
        bad_files.append({
            "path": file_path,
            "reason": str(e)
        })

# --- 최종 결과 출력 ---
print("\n" + "="*50)
print("           무결성 검사 결과")
print("="*50)

if not bad_files:
    print(f"✅ 총 {len(image_paths)}개의 파일을 모두 성공적으로 확인했습니다.")
    print("손상되거나 읽을 수 없는 이미지가 발견되지 않았습니다.")
else:
    print(f"⚠️ 총 {len(image_paths)}개의 파일 중 {len(bad_files)}개의 문제 파일을 찾았습니다.")
    print("\n--- 🗑️ 문제 파일 목록 ---")
    for item in bad_files:
        print(f"파일: {item['path']}")
        print(f"  원인: {item['reason']}")
        print("-" * 20)
    print("\n위 파일들은 모델 학습 시 오류를 유발할 수 있으므로 삭제하거나 수정해야 합니다.")

print("="*50)

이미지 무결성 검사를 시작합니다...
대상 경로: /Users/ihanjo/Documents/미니 프로젝트/한국음식 예측/데이터/data/kfood


파일 검사 중:  45%|████▌     | 67921/150611 [01:16<01:21, 1017.91it/s]/opt/anaconda3/envs/stenv/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
파일 검사 중: 100%|██████████| 150611/150611 [02:38<00:00, 950.76it/s] 


           무결성 검사 결과
⚠️ 총 150611개의 파일 중 1개의 문제 파일을 찾았습니다.

--- 🗑️ 문제 파일 목록 ---
파일: ../데이터/data/kfood/.DS_Store
  원인: cannot identify image file '../데이터/data/kfood/.DS_Store'
--------------------

위 파일들은 모델 학습 시 오류를 유발할 수 있으므로 삭제하거나 수정해야 합니다.
